# Embryo Data Query

This notebook queries embryo-related data from multiple databases for a given prontuario/PIN.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_congelamentos_embrioes
  - view_descongelamentos_embrioes
  - view_embrioes_congelados
- **Huntington Data Lake** (huntington_data_lake.duckdb):
  - gold.clinisys_embrioes
  - gold.embryoscope_embrioes
  - gold.embryoscope_clinisys_combined
  - gold.embryoscope_treatment (from embryoscope_embrioes)
  - silver.planilha_embriologia
  - gold.planilha_embryoscope_combined


In [32]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Database Connections


In [33]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Query Functions - Clinisys Tables


In [34]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [35]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    return df

def query_congelamentos_embrioes(conn, prontuario):
    """Query view_congelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_embrioes")
    return df

def query_descongelamentos_embrioes(conn, prontuario):
    """Query view_descongelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_embrioes")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    return df

def query_congelamentos_ovulos(conn, prontuario):
    """Query view_congelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_ovulos")
    return df

def query_descongelamentos_ovulos(conn, prontuario):
    """Query view_descongelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_ovulos")
    return df

def query_ovulos_congelados(conn, prontuario):
    """Query view_ovulos_congelados for a specific prontuario"""
    print(f"\n=== VIEW_OVULOS_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_ovulos_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_ovulos_congelados")
    return df


## Query Functions - Huntington Data Lake Tables


In [36]:
def query_clinisys_embrioes(conn, prontuario):
    """Query gold.clinisys_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.CLINISYS_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.clinisys_embrioes 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, oocito_id_micromanipulacao DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.clinisys_embrioes")
    return df

def query_embryoscope_embrioes(conn, prontuario):
    """Query gold.embryoscope_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.embryoscope_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY embryo_KIDDate DESC, treatment_TreatmentName DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_embrioes")
    return df

def query_embryoscope_clinisys_combined(conn, prontuario):
    """Query gold.embryoscope_clinisys_combined for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_CLINISYS_COMBINED ===")
    query = f"""
    SELECT * FROM gold.embryoscope_clinisys_combined 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, oocito_id_micromanipulacao DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_clinisys_combined")
    return df


In [37]:
def query_planilha_embriologia_fresh(conn, prontuario):
    """Query silver.planilha_embriologia for a specific prontuario"""
    print(f"\n=== SILVER.PLANILHA_EMBRIOLOGIA_FRESH ===")
    query = f"""
    SELECT * FROM silver.planilha_embriologia_fresh 
    WHERE prontuario = {prontuario}
    -- ORDER BY "DATA DA PUNÇÃO" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.planilha_embriologia_fresh")
    return df

def query_planilha_embriologia_fet(conn, prontuario):
    """Query silver.planilha_embriologia for a specific prontuario"""
    print(f"\n=== SILVER.PLANILHA_EMBRIOLOGIA_FET ===")
    query = f"""
    SELECT * FROM silver.planilha_embriologia_fet 
    WHERE prontuario = {prontuario}
    -- ORDER BY "DATA DA FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.planilha_embriologia_fresh")
    return df

def query_planilha_embryoscope_combined(conn, prontuario):
    """Query gold.planilha_embryoscope_combined for a specific prontuario"""
    print(f"\n=== GOLD.PLANILHA_EMBRYOSCOPE_COMBINED ===")
    query = f"""
    SELECT * FROM gold.planilha_embryoscope_combined 
    WHERE "micro_prontuario" = {prontuario}
    ORDER BY "oocito_id" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.planilha_embryoscope_combined")
    return df


In [38]:
def query_redlara_unified(conn, prontuario):
    """Query silver.redlara_unified for a specific prontuario"""
    print(f"\n=== silver.redlara_unified ===")
    query = f"""
    SELECT * FROM silver.redlara_unified
    where prontuario = {prontuario}
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.redlara_unified")
    return df


def query_redlara_planilha_combined(conn, prontuario):
    """Query gold.redlara_planilha_combined for a specific prontuario"""
    print(f"\n=== gold.redlara_planilha_combined ===")
    query = f"""
    SELECT * FROM gold.redlara_planilha_combined
    where prontuario = {prontuario}
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.redlara_planilha_combined")
    return df


## Main Query Function


In [39]:
def query_embryo_data(prontuario):
    """
    Query all embryo-related data for a given prontuario/PIN.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results
    """
    print(f"\n{'='*80}")
    print(f"QUERYING EMBRYO DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_congelamentos_embrioes = query_congelamentos_embrioes(clinisys_con, prontuario)
        df_descongelamentos_embrioes = query_descongelamentos_embrioes(clinisys_con, prontuario)
        df_embrioes_congelados = query_embrioes_congelados(clinisys_con, prontuario)
        df_congelamentos_ovulos = query_congelamentos_ovulos(clinisys_con, prontuario)
        df_descongelamentos_ovulos = query_descongelamentos_ovulos(clinisys_con, prontuario)
        df_ovulos_congelados = query_ovulos_congelados(clinisys_con, prontuario)
        
        # Query huntington data lake tables
        df_clinisys_embrioes = query_clinisys_embrioes(huntington_con, prontuario)
        df_embryoscope_embrioes = query_embryoscope_embrioes(huntington_con, prontuario)
        df_embryoscope_clinisys_combined = query_embryoscope_clinisys_combined(huntington_con, prontuario)
        # df_embryoscope_treatment = query_embryoscope_treatment(huntington_con, prontuario)
        df_planilha_embriologia_fet = query_planilha_embriologia_fet(huntington_con, prontuario)
        df_planilha_embriologia_fresh = query_planilha_embriologia_fresh(huntington_con, prontuario)
        df_planilha_embryoscope_combined = query_planilha_embryoscope_combined(huntington_con, prontuario)
        df_redlara_unified = query_redlara_unified(huntington_con, prontuario)
        df_redlara_planilha_combined = query_redlara_planilha_combined(huntington_con, prontuario)

        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_congelamentos_embrioes: {len(df_congelamentos_embrioes)} records")
        print(f"- view_descongelamentos_embrioes: {len(df_descongelamentos_embrioes)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes_congelados)} records")
        print(f"- view_congelamentos_ovulos: {len(df_congelamentos_ovulos)} records")
        print(f"- view_descongelamentos_ovulos: {len(df_descongelamentos_ovulos)} records")
        print(f"- view_ovulos_congelados: {len(df_ovulos_congelados)} records")
        print(f"- gold.clinisys_embrioes: {len(df_clinisys_embrioes)} records")
        print(f"- gold.embryoscope_embrioes: {len(df_embryoscope_embrioes)} records")
        print(f"- gold.embryoscope_clinisys_combined: {len(df_embryoscope_clinisys_combined)} records")
        print(f"- silver.planilha_embriologia_fet: {len(df_planilha_embriologia_fet)} records")
        print(f"- silver.planilha_embriologia_fresh: {len(df_planilha_embriologia_fresh)} records")
        print(f"- gold.planilha_embryoscope_combined: {len(df_planilha_embryoscope_combined)} records")
        print(f"- silver.redlara_unified: {len(df_redlara_unified)} records")
        print(f"- gold.redlara_planilha_combined: {len(df_redlara_planilha_combined)} records")

        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'congelamentos_embrioes': df_congelamentos_embrioes,
            'descongelamentos_embrioes': df_descongelamentos_embrioes,
            'embrioes_congelados': df_embrioes_congelados,
            'congelamentos_ovulos': df_congelamentos_ovulos,
            'descongelamentos_ovulos': df_descongelamentos_ovulos,
            'ovulos_congelados': df_ovulos_congelados,
            'clinisys_embrioes': df_clinisys_embrioes,
            'embryoscope_embrioes': df_embryoscope_embrioes,
            'embryoscope_clinisys_combined': df_embryoscope_clinisys_combined,
            'planilha_embriologia_fet': df_planilha_embriologia_fet,
            'planilha_embriologia_fresh': df_planilha_embriologia_fresh,
            'planilha_embryoscope_combined': df_planilha_embryoscope_combined,
            'redlara_unified': df_redlara_unified,
            'redlara_planilha_combined': df_redlara_planilha_combined,
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Example Usage


In [40]:
# Example usage
prontuario = 827681

# Query all embryo data
results = query_embryo_data(prontuario)



QUERYING EMBRYO DATA FOR PRONTUARIO: 827681
Query executed at: 2026-02-04 10:48:22.983468

=== VIEW_MICROMANIPULACAO ===
Found 3 records in view_micromanipulacao

=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 31 records in view_micromanipulacao_oocitos

=== VIEW_TRATAMENTOS ===
Found 1 records in view_tratamentos

=== VIEW_CONGELAMENTOS_EMBRIOES ===
Found 3 records in view_congelamentos_embrioes

=== VIEW_DESCONGELAMENTOS_EMBRIOES ===
Found 2 records in view_descongelamentos_embrioes

=== VIEW_EMBRIOES_CONGELADOS ===
Found 10 records in view_embrioes_congelados

=== VIEW_CONGELAMENTOS_OVULOS ===
Found 1 records in view_congelamentos_ovulos

=== VIEW_DESCONGELAMENTOS_OVULOS ===
Found 2 records in view_descongelamentos_ovulos

=== VIEW_OVULOS_CONGELADOS ===
Found 8 records in view_ovulos_congelados

=== GOLD.CLINISYS_EMBRIOES ===
Found 31 records in gold.clinisys_embrioes

=== GOLD.EMBRYOSCOPE_EMBRIOES ===
Found 31 records in gold.embryoscope_embrioes

=== GOLD.EMBRYOSCOPE_CLINISYS_COMBIN

## Display Results

### 1. Micromanipulacao


In [41]:
print("=== MICROMANIPULACAO DATA ===")
micromanipulacao_data = results['micromanipulacao']
print(f"Records: {len(micromanipulacao_data)}")
if not micromanipulacao_data.empty:
    display(micromanipulacao_data)
else:
    print("No micromanipulacao data found.")


=== MICROMANIPULACAO DATA ===
Records: 3


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,codigo_congelamento_semen,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,abstinencia_recomendada,recepcao_ovulos,CicloDoadora,IdadeDoadora,checagem_d4,Denud_DL,checagem_fertilizacao,checagem_d2,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,sangue_cateter_interno,dificuldade_transf,retorno_embrioes,vezes_retorno,sangue_cateter_externo,ICSIDescongelados,CicloDescongelamento,pgd_oocito,pgd_d3,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,mini_gradiente,data_procedimento_2,tecnica_injetados_2,motilidade_2,morfologia_injetados_2,unidade_id,data_microtese,data_microtese_2,validade7,responsavel_denud_id,profissional_fertilizacao_id,profissional_d2_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,validade11,materiais_insumos,local_coleta,fonte_outros,gradiente_descontinuo_2,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,mini_gradiente_2,local_coleta_2,fonte_outros_2,observacoes_resumo_ciclos,viscosidade_2,volume_amostra_utilizada_2,morfologia_pre_2,morfologia_pos_2,zymot_2,validade20,gradiente,lote21,validade21,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,c_ps1,d_ps1,conc_ps1,diametro_minimo_pse,diametro_medio_pse,diametro_maximo_pse,tubulos_avaliados_pse,peso_total_pse,peso_microtese_pse,peso_biopsia_pse,diametro_minimo_psd,diametro_medio_psd,diametro_maximo_psd,tubulos_avaliados_psd,peso_total_psd,peso_microtese_psd,peso_biopsia_psd,responsavel_transfer_id,MorfologiaAlterada,EstadoSptz,CongelamentoSemenHom,VialsDescongeladasHom,CongelamentoSemenHet,VialsDescongeladasHet,check_gradiente_ps,concentrado,check_swimup,check_lavagem,volume_ps2,MotivoCongelamentoOvulos,concentracao_progressivos_pos,c_ps2,d_ps2,conc_ps2,comentarios_ps,responsavel_ps,fonte2,inicio_ps2,termino_ps2,volume_ps3,concentracao_progressivos_pre_2,c_ps3,d_ps3,conc_ps3,diametro_minimo_pse2,diametro_medio_pse2,diametro_maximo_pse2,tubulos_avaliados_pse2,peso_total_pse2,peso_microtese_pse2,peso_biopsia_pse2,diametro_minimo_psd2,diametro_medio_psd2,diametro_maximo_psd2,tubulos_avaliados_psd2,peso_total_psd2,peso_microtese_psd2,peso_biopsia_psd2,timelapse,EstadoSptz2,codigo_congelamento_semen_2,CongelamentoSemenHom2,VialsDescongeladasHom2,CongelamentoSemenHet2,VialsDescongeladasHet2,check_gradiente_ps2,concentrado_2,check_swimup2,check_lavagem2,volume_ps4,concentracao_progressivos_pos_2,c_ps4,d_ps4,conc_ps4,comentarios_ps2,responsavel_ps2,transferidos,cateter,lote_cateter,validade_cateter,motivo_nao_transferencia,Transfer_D5,medico_responsavel_id,azoospermia,cod1_crio,cod2_crio,tanque,cane,tecnica,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,responsavel_biopsia,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,23412,ED401/24,827681,44,44,2024-11-24,None,12:30,12:50,None,31,None,None,"Sim, descongelada",None,None,None,None,08:00,None,08:00,08:00,08:00,08:00,None,8,None,None,None,None,None,Sim,DO665/24,1,None,None,2024-11-24,ICSI,08 espermatozoides móveis progressivos\r\n,08 espermatozoides normais\r\n,1,None,None,None,None,None,0000000001,<NA>,<NA>,<NA>,

### 2. Micromanipulacao Oocitos


In [42]:
print("=== MICROMANIPULACAO OOCITOS DATA ===")
oocitos_data = results['micromanipulacao_oocitos']
print(f"Records: {len(oocitos_data)}")
if not oocitos_data.empty:
    display(oocitos_data)
else:
    print("No micromanipulacao oocitos data found.")


=== MICROMANIPULACAO OOCITOS DATA ===
Records: 31


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,ComentariosAntes,Embriologista,PI,TCD,AH,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,GD1,OocitoDoado,ICSI,OrigemOocito,InseminacaoOocito,NClivou_D2,NCelulas_D2,Frag_D2,Blastomero_D2,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,GD2,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,embryo_number
0,195968,20500,Não,MII,None,None,3613,None,Descartado,None,None,None,None,2PN,None,Fresco,None,Não,None,Descongelado OR,IMSI,None,None,None,None,Sim,4,1,Simétricos,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,7f18893220e6ea837cb065bef0f72479,2025-07-21 21:40:02,1
1,195969,20500,Não,MII,None,None,3613,None,Descartado,None,None,None,None,1CP,None,Fresco,None,Não,None,Descongelado OR,IMSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1c161cd96fc5dd5cfdeb4e3f63ca7883,2025-07-21 21:40:02,2
2,195970,20500,Não,MII,None,None,3613,None,Descartado,None,None,None,None,2PN,None,Fresco,None,Não,None,Descongelado OR,IMSI,None,None,None,None,Sim,8,1,Simétricos,None,None,None,None,None,None,Sim,None,Blastocisto Grau 2,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,bbbda63f4967dfaa66a1cb2fcfa9b519,2025-07-21 21:40:02,3
3,195971,20500,Não,MII,None,None,3613,None,Descartado,None,None,None,None,2PN,None,Fresco,None,Não,None,Descongelado OR,IMSI,None,None,None,None,Sim,5,1,Simétricos,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,76b31878169350b73cb0ca96dc095fd9,2025-07-21 21:40:02,4
4,195972,20500,Não,MII,None,None,3613,None,Descartado,None,None,None,None,2PN,None,Fresco,None,Não,None,Descongelado OR,IMSI,None,None,None,None,Sim,4,1,Simétricos,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,05704bca34848ddde91a8961d148da80,2025-07-21 21:40:02,5
5,195973,20500,Não,MII,None,None,3613,3548,Criopreservado,Sim,Sim,Aneuploide,PGT-A,2PN,None,Fresco,None,Não,None,Descongelado OR,IMSI,None,None,None,None,Sim,5,2,Simétricos,Sim,None,None,None,None,None,Sim,None,Blastocisto Grau 2,None,None,Sim,None,Blastocisto Grau 4,C,C,None,None,None,None,None,2.58,Sim,a4aab2422758811b1bba97ef5a50b6ba,2025-07-21 21:40:02,6
6,195974,20500,Não,MII,None,None,3613,3548,Criopreservado,Sim,Sim,Aneuploide,PGT-A,2PN,None,Fresco,None,Não,None,Descongelado OR,IMSI,None,None,None,None,Sim,8,1,Simétricos,Não,None,None,None,None,None,Sim,None,Blastocisto Grau 2,None,None,Sim,None,Blastocisto Grau 5,A,B,None,None,None,None,None,7.51,Sim,75bc55b81d8fa15bedd3d43f1ac22ca4,2025-07-21 21:40:02,7
7,195975,20500,Não,MII,None,None,3613,3548,Criopreservado,Sim,Sim,Euploide,PGT-A,2PN,None,Fresco,None,Não,None,Descongelado OR,IMSI,None,None,None,None,Sim,4,2,Simétricos,Sim,None,None,None,None,None,Sim,None,Blastocisto Grau 2,None,None,Sim,None,Blastocisto Grau 5,B,B,None,None,None,None,None,7.4,Sim,977223c7d5429f1db5ec8a76f2fe8d56,2025-07-21 21:40:02,8
8,195976,20500,Não,MII,None,None,3613,None,Descartado,None,None,None,None,2PN,None,Fresco,None,Não,None,Descongelado OR,ICSI,None,None,None,None,Sim,5,2,Simétricos,None,None,None,None,None,None,Sim,None,Blastocisto Grau 1,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,27676ae96bfc2536a81396c5457da68e,2025-07-21 21:40:02,9
9,195977,20500,Não,MII,None,None,3613,3548,Criopreservado,Sim,Sim,Euploide,PGT-A,2PN,None,Fresco,None,Não,None,Descongelado OR,IMSI,None,None,None,None,Sim,8,1,None,Sim,None,None,None,None,None,Sim,None,Blastocisto Grau 5,A,A,None,None,N

### 3. Tratamentos


In [43]:
print("=== TRATAMENTOS DATA ===")
tratamentos_data = results['tratamentos']
print(f"Records: {len(tratamentos_data)}")
if not tratamentos_data.empty:
    display(tratamentos_data)
else:
    print("No tratamentos data found.")


=== TRATAMENTOS DATA ===
Records: 1


,id,prontuario,unidade,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,hora_procedimento,tipo_procedimento,motivo_congelamento,tipo_inducao,protocolo,tipo_ciclo_congelado,tipo_preparo_endometrial,num_congelados,num_transferir,origem_ovulo,status_ovulo,prontuario_doadora,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,material_heterologo,utero_substituicao,prontuario_genitores,prontuario_uter_substituicao,doacao_ovulos,destino_doacao_ovulos,prontuario_receptora,complemento_receptora,doacao_embrioes,era,emma,alice,matrice_lab,imunohistoquimica,outros_biopsia,consentimento,financeiro,embryoscope,pgt,era_auxiliar,recomendacao_transferencia,resultado_era,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,fator_infertilidade3,fsh_exame,foliculos_antrais,data_histeroscopia,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,ropa,data_inicio_agonista,gnrh,fsh_agonista,lh_agonista,outros_medicamentos,gonal,menogon,menopur,pergoveris,elonva,serofene_clomid_indux,luveris,puregon,letrozole,fostimon,merional,rekovelle,descricao_outras_medicacoes,dose_total_fsh,dose_total_lh,dose_total_fsh_lh,coorte_e2,coorte_aco,coorte_progesterona,coq10,hormonal_combinado,coorte_testosterona,agonista_coorte,coorte_antagonista,coorte_dhea,utilizado_hcg,motivo_cancelamento_hcg,data_hcg,horario_planejado_hcg,horario_real_hcg,ovidrel,gonapeptyl,dose_gonapeptyl,outras_complicacoes_hcg,dose_outras_complicacoes,lupron,outros_hcg,descricao_outros_hcg,data_inicio_progesterona,p4_inicio_progesterona,estradiol_fase_lutea,progesterona_fase_lutea,progesterona_oral,progesterona_vaginal,progesterona_injetavel,combinacao_oral_vaginal,nenhuma_progesterona,hcg_progesterona,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,observacoes_descongelamento,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,p4_transferencia,motivo_nao_transferir,motivo_cancelamento_tratamento,ligacao_enfermagem,beta_valor,beta_valor2,data_ultrassom1,data_ultrassom2,num_sacos_gestacionais,tsh,data_ultrassom1_contato,descricao_ultrassom1,bcf_embrião1_status,bcf_embrião1_fonte,data_ultrassom2_contato,descricao_ultrassom2,data_ultrassom3_contato,descricao_ultrassom3,resultado_tratamento,analise_genetica,resultado_analise_genetica,motivo_perda_aborto,data_nascimento_bebes,obstetra,nome_bebe1,sexo_bebe1,peso_bebe1,altura_bebe1,idade_gestacional_bebe1,via_parto_bebe1,viabilidade_bebe1,anomalias_bebe1,nome_bebe2,sexo_bebe2,peso_bebe2,altura_bebe2,idade_gestacional_bebe2,via_parto_bebe2,viabilidade_bebe2,anomalias_bebe2,nome_bebe3,sexo_bebe3,peso_bebe3,altura_bebe3,idade_gestacional_bebe3,via_parto_bebe3,viabilidade_bebe3,anomalias_bebe3,nome_bebe4,sexo_bebe4,peso_bebe4,altura_bebe4,idade_gestacional_bebe4,via_parto_bebe4,viabilidade_bebe4,anomalias_bebe4,observacoes_bebes,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp,bmi,previous_et,previous_et_od
0,28059,827681,1,44,44,esposa,1,2024-06-21,None,Ciclo a Fresco FIV,None,Convencional,None,None,Hormonal,None,None,Heterólogo,Fresco,786406.0,Homólogo,Sêmen,Fresco,None,None,Sim,827681.0,836751.0,Não,None,NaN,None,Não,None,None,None,None,None,None,None,None,None,PGT-A,None,None,None,None,None,None,None,None,None,0.0,0.0,None,NaT,NaN,NaN,None,Outros,None,None,None,None,None,2024-06-21,None,esposa,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Não utilizado,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaT,None,None,None,None,No transfer,NaT,None,None,None,Criopreservação total,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No transfer,Não,None,N

### 4. Congelamentos Embrioes


In [44]:
print("=== CONGELAMENTOS EMBRIOES DATA ===")
congelamentos_data = results['congelamentos_embrioes']
print(f"Records: {len(congelamentos_data)}")
if not congelamentos_data.empty:
    display(congelamentos_data)
else:
    print("No congelamentos embrioes data found.")


=== CONGELAMENTOS EMBRIOES DATA ===
Records: 3


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,condicoes_amostra,empresa_transporte,clinica_origem,responsavel_recebimento,responsavel_recebimento_data,responsavel_armazenamento,responsavel_armazenamento_data,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Ovulo,D2,D3,D4,D5,D6,D7,rack,rack2,rack3,rack4,Observacoes,BiologoResponsavel,BiologoFIV,BiologoFIV2,status_financeiro,responsavel_congelamento_d5,responsavel_checagem_d5,responsavel_congelamento_d6,responsavel_checagem_d6,responsavel_congelamento_d7,responsavel_checagem_d7,hash,extraction_timestamp
0,27413,E3168/24,1,827681,esposa,2024-11-30,1900-01-01 12:30:00,ED401/24,Não,None,None,None,<NA>,NaT,<NA>,NaT,4,4,E21791,None,None,None,Vitrificação,NaN,NaN,NaN,NaN,NaN,1.0,NaN,None,NaN,NaN,NaN,None,<NA>,Dayane Reis,Beatriz Aiello,Cobrar,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,c8db51c3cca0f81cab32db4b87e42309,2025-09-15 21:10:57
1,25886,E1652/24,1,827681,esposa,2024-06-26,1900-01-01 17:00:00,ED209/24/E21840,Não,None,None,None,<NA>,NaT,<NA>,NaT,2,2,M9954/E21840,None,None,None,Vitrificação,NaN,NaN,NaN,NaN,1.0,NaN,NaN,None,NaN,NaN,NaN,None,<NA>,Larissa Jordão,Thiago Lotfi,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,78cce91a16cb5b7fbacaff8b03e72655,2025-07-21 21:38:47
2,25405,E1175/24,1,827681,esposa,2024-05-11,1900-01-01 14:00:00,1447/24,Não,None,None,None,<NA>,NaT,<NA>,NaT,4,4,E21034,None,None,None,Vitrificação,NaN,NaN,NaN,NaN,1.0,1.0,NaN,None,NaN,NaN,NaN,None,<NA>,Dayane Reis,Laura Belomo,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,8b1b59e0663ef16cc981a3ae586bcbac,2025-07-21 21:38:47


### 5. Descongelamentos Embrioes


In [45]:
print("=== DESCONGELAMENTOS EMBRIOES DATA ===")
descongelamentos_data = results['descongelamentos_embrioes']
print(f"Records: {len(descongelamentos_data)}")
if not descongelamentos_data.empty:
    display(descongelamentos_data)
else:
    print("No descongelamentos embrioes data found.")


=== DESCONGELAMENTOS EMBRIOES DATA ===
Records: 2


,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,Transferencia,DataTransferencia,Prateleira,Incubadora,transferidos_transferencia,cateter_transferencia,lote_transferencia,validade_transferencia,intercorrencia_transferencia,sangue_interno_transferencia,sangue_externo_transferencia,retorno_transferencia,vezes_retorno_transferencia,Transfer_D5,responsavel_transferencia,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp
0,18866,2083/25,1,827681,0,2024-11-30,2025-09-13,ED401/24,E21791,E3168/24,AH,6,1,Desvitrificação,<NA>,2025-09-13,<NA>,13.6,1,SYDNEY,None,None,None,Não,Não,Não,0,None,4005.0,None,None,None,2be792ab430bbcb845d88642025e3224,2025-09-19 19:03:08
1,16635,2760/24,1,827681,0,2024-11-30,2024-12-13,ED401/24,E21791,E3168/24,AH,6,1,DESVITRIFICAÇÃO,<NA>,2024-12-13,<NA>,13.9,1,SYDNEY,None,None,None,Não,Não,Não,None,None,3719.0,casal tem como útero de substituição: 836.751,None,None,badb81c73f97accc9fcbea5c23d93dee,2025-07-21 21:38:51


### 6. Embrioes Congelados


In [46]:
print("=== EMBRIOES CONGELADOS DATA ===")
embrioes_congelados_data = results['embrioes_congelados']
print(f"Records: {len(embrioes_congelados_data)}")
if not embrioes_congelados_data.empty:
    display(embrioes_congelados_data)
    # display(embrioes_congelados_data[embrioes_congelados_data['id_descongelamento']!=0])
else:
    print("No embrioes congelados data found.")


=== EMBRIOES CONGELADOS DATA ===
Records: 10


,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,cores,embriao,doado,lote,celulas,qualidade,qualidade_recongelamento,comentarios,comentarios_descongelamento,ah,dia2,maturidade,dia2_2,dia2_3,dia3,dia3_2,dia3_3,dia4,dia4_2,dia4_3,dia4_4,gd,gd2,gd3,dia5,dia5_2,dia5_3,dia5_4,dia6,dia6_2,dia6_3,dia6_4,dia7,dia7_2,dia7_3,dia7_4,clivou_d2,clivou_d3,clivou_d4,clivou_d5,clivou_d6,clivou_d7,transferidos,pgd,resultado_pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,observacao,destino,hash,extraction_timestamp
0,79828,228982,27413,18866,827681,P7,E21791,<NA>,#7,None,None,None,Blastocisto Grau 3 - B - C,<NA>,None,"EUPLOIDE, Feminino",Sim,None,None,None,None,None,None,None,None,None,None,None,Sim,None,None,None,None,None,None,None,Blastocisto Grau 3,B,C,None,None,None,None,<NA>,None,None,None,None,None,Transferido,Sim,Euploide,"FEMININO, EUPLOIDE",Sim,Euploide,3.1,D6,6.96,AH,6,E21791,None,<NA>,be25927374af2cb4d3c805e8fe152bae,2025-09-15 21:11:13
1,79827,228981,27413,16635,827681,P6,E21791,<NA>,#6,None,None,None,Blastocisto Grau 5 - A - B,<NA>,None,"EUPLOIDE, Feminino",Sim,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Blastocisto Grau 5,A,B,None,None,None,None,<NA>,None,None,None,None,None,Transferido,Sim,Euploide,"FEMININO, EUPLOIDE",Sim,Euploide,6.4,D6,2.21,AH,6,E21791,None,<NA>,2759ee85046c3d1d2ae25447777fedab,2025-07-21 21:38:57
2,79826,228980,27413,0,827681,P5,E21791,<NA>,#5,None,None,None,Blastocisto Grau 4 - B - B,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,"XY, Trissomia parcial chr 1p Masculino, Aneuploide",Sim,Aneuploide,4.5,D6,2.0,AH,6,E21791,MANTER,<NA>,0fb3537290bfeef5f5763ad603fc31fd,2025-07-21 21:38:57
3,79825,228979,27413,0,827681,P4,E21791,<NA>,#4,None,None,None,Blastocisto Grau 5 - B - C,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,Descartado,None,None,"XX, Monossomia chr 4, Monossomia chr 16 Feminino, AneuploideXX, Monossomia chr 4, Monossomia chr 16 Feminino, Aneuploide",Sim,Aneuploide,3.1,D6,8.78,AH,6,E21791,None,<NA>,ed2a6544d6bda044a499d373137aded4,2025-07-21 21:38:57
4,74303,203312,25886,0,827681,#11,M9954/E21840,<NA>,#11,None,None,None,Blastocisto Grau 5 - B - B,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,Masculino,Sim,Euploide,5.6,D5,7.61,AK4,1,M9954/E21840,None,<NA>,f342f9cc3525f79490671e6f863d3d8f,2025-07-21 21:38:57
5,74302,203307,25886,0,827681,#6,M9954/E21840,<NA>,#6,None,None,None,Blastocisto Grau 4 - B - C,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,"Feminino, Trissomia chr 13 e chr 21",Sim,Mosaico baixo grau,4.7,D5,6.95,AK4,1,M9954/E21840,None,<NA>,ef80993ae137191333801aadba310525,2025-07-21 21:38:57
6,72528,195977,25405,0,827681,P10,E21034,<NA>,#10,None,None,None,Blastocisto Grau 5 - A - A,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,Normal/Euplóide Masculino,Sim,Euploide,8.7,D5,6.91,BETA,6,E21034,None,<NA>,a2b3e313c478905923f487d0d7a8622c,2025-07-21 21:38:57
7,72527,195975,25405,0,827681,P8,E21034,<NA>,#8,None,None,None,Blastocisto Grau 5 - B - B,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<N

### 7. Clinisys Embrioes


In [47]:
print("=== CLINISYS EMBRIOES DATA ===")
clinisys_embrioes_data = results['clinisys_embrioes']
print(f"Records: {len(clinisys_embrioes_data)}")
if not clinisys_embrioes_data.empty:
    display(clinisys_embrioes_data.head(30))
else:
    print("No clinisys embrioes data found.")


=== CLINISYS EMBRIOES DATA ===
Records: 31


,oocito_id,oocito_id_micromanipulacao,oocito_TCD,oocito_embryo_number,oocito_OrigemOocito,micro_numero_caso,micro_prontuario,micro_Data_DL,micro_data_procedimento,micro_oocitos,trat1_id,trat1_tentativa,trat1_tipo_procedimento,trat1_data_inicio_inducao,trat1_motivo_nao_transferir,trat1_data_transferencia,trat1_resultado_tratamento,trat1_fator_infertilidade1,trat1_origem_ovulo,trat1_bmi,trat1_previous_et,trat1_previous_et_od,emb_cong_id,emb_cong_qualidade,emb_cong_transferidos,emb_cong_colapso,cong_em_id,cong_em_CodCongelamento,cong_em_Data,descong_em_id,descong_em_CodDescongelamento,descong_em_DataTransferencia,descong_em_DataCongelamento,descong_em_DataDescongelamento,trat2_id,trat2_tentativa,trat2_tipo_procedimento,trat2_data_inicio_inducao,trat2_motivo_nao_transferir,trat2_data_transferencia,trat2_resultado_tratamento,trat2_bmi,trat2_previous_et,trat2_previous_et_od
0,228976,23412,Descartado,1,Descongelado OR,ED401/24,827681,2024-11-24,2024-11-24,8,<NA>,None,None,NaT,None,NaT,None,None,None,NaN,<NA>,<NA>,<NA>,None,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>
1,228977,23412,Descartado,2,Descongelado OR,ED401/24,827681,2024-11-24,2024-11-24,8,<NA>,None,None,NaT,None,NaT,None,None,None,NaN,<NA>,<NA>,<NA>,None,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>
2,228978,23412,Descartado,3,Descongelado OR,ED401/24,827681,2024-11-24,2024-11-24,8,<NA>,None,None,NaT,None,NaT,None,None,None,NaN,<NA>,<NA>,<NA>,None,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>
3,228979,23412,Criopreservado,4,Descongelado OR,ED401/24,827681,2024-11-24,2024-11-24,8,<NA>,None,None,NaT,None,NaT,None,None,None,NaN,<NA>,<NA>,79825,Blastocisto Grau 5 - B - C,Descartado,"XX, Monossomia chr 4, Monossomia chr 16 Feminino, AneuploideXX, Monossomia chr 4, Monossomia chr 16 Feminino, Aneuploide",27413,E3168/24,2024-11-30,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>
4,228980,23412,Criopreservado,5,Descongelado OR,ED401/24,827681,2024-11-24,2024-11-24,8,<NA>,None,None,NaT,None,NaT,None,None,None,NaN,<NA>,<NA>,79826,Blastocisto Grau 4 - B - B,None,"XY, Trissomia parcial chr 1p Masculino, Aneuploide",27413,E3168/24,2024-11-30,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>
5,228981,23412,Criopreservado,6,Descongelado OR,ED401/24,827681,2024-11-24,2024-11-24,8,<NA>,None,None,NaT,None,NaT,None,None,None,NaN,<NA>,<NA>,79827,Blastocisto Grau 5 - A - B,Transferido,"FEMININO, EUPLOIDE",27413,E3168/24,2024-11-30,16635,2760/24,2024-12-13,2024-11-30,2024-12-13,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>
6,228982,23412,Criopreservado,7,Descongelado OR,ED401/24,827681,2024-11-24,2024-11-24,8,<NA>,None,None,NaT,None,NaT,None,None,None,NaN,<NA>,<NA>,79828,Blastocisto Grau 3 - B - C,Transferido,"FEMININO, EUPLOIDE",27413,E3168/24,2024-11-30,18866,2083/25,2025-09-13,2024-11-30,2025-09-13,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>
7,228983,23412,Descartado,8,Descongelado OR,ED401/24,827681,2024-11-24,2024-11-24,8,<NA>,None,None,NaT,None,NaT,None,None,None,NaN,<NA>,<NA>,<NA>,None,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>
8,203302,21127,Descartado,1,Fresco OR,ED209/24,827681,2024-06-21,2024-06-21,11,28059,1,Ciclo a Fresco FIV,2024-06-21,Criopreservação total,NaT,No transfer,Outros,Heterólogo,NaN,0,0,<NA>,None,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>
9,203303,21127,Descartado,2,Fresco OR,ED209/24,827681,2024-06-21,2024-06-21,11,28059,1,Ciclo a Fresco FIV,2024-06-21,Criopreservação total,NaT,No transfer,Outros,Heterólogo,NaN,0,0,<NA>,None,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>


In [48]:
# clinisys_embrioes_data[clinisys_embrioes_data['oocito_id'].isin([184181])]

In [49]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'oocito_id_micromanipulacao', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento'
]

# clinisys_embrioes_data.loc[clinisys_embrioes_data['oocito_embryo_number'].isin([33,35]), colunas]

### 8. Embryoscope Embrioes


In [50]:
print("=== EMBRYOSCOPE EMBRIOES DATA ===")
embryoscope_embrioes_data = results['embryoscope_embrioes']
print(f"Records: {len(embryoscope_embrioes_data)}")
if not embryoscope_embrioes_data.empty:
    display(embryoscope_embrioes_data)
else:
    print("No embryoscope embrioes data found.")


=== EMBRYOSCOPE EMBRIOES DATA ===
Records: 31


,prontuario,patient_PatientID,patient_PatientIDx,patient_FirstName,patient_LastName,patient_DateOfBirth,patient_YearOfBirth,patient_unit_huntington,treatment_TreatmentName,embryo_EmbryoID,embryo_EmbryoDate,embryo_embryo_number,embryo_Position,embryo_InstrumentNumber,embryo_WellNumber,embryo_FertilizationTime,embryo_FertilizationMethod,embryo_EmbryoFate,embryo_Description,embryo_EmbryoDescriptionID,embryo_KIDScore,embryo_KIDDate,embryo_KIDVersion,embryo_KIDUser,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Value_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Value_EVEN2,embryo_Timestamp_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Value_EVEN4,embryo_Timestamp_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Value_EVEN8,embryo_Timestamp_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Value_FRAG2,embryo_Timestamp_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Value_FRAG4,embryo_Timestamp_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Value_FRAG8,embryo_Timestamp_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Value_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Value_ICM,embryo_Timestamp_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Value_Line,embryo_Timestamp_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Value_MN2Type,embryo_Timestamp_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Value_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Value_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Value_PN,embryo_Timestamp_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Value_Pulsing,embryo_Timestamp_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Value_Strings,embryo_Timestamp_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Value_TE,embryo_Timestamp_TE,embryo_Name_t2,embryo_Time_t2,embryo_Value_t2,embryo_Timestamp_t2,embryo_Name_t3,embryo_Time_t3,embryo_Value_t3,embryo_Timestamp_t3,embryo_Name_t4,embryo_Time_t4,embryo_Value_t4,embryo_Timestamp_t4,embryo_Name_t5,embryo_Time_t5,embryo_Value_t5,embryo_Timestamp_t5,embryo_Name_t6,embryo_Time_t6,embryo_Value_t6,embryo_Timestamp_t6,embryo_Name_t7,embryo_Time_t7,embryo_Value_t7,embryo_Timestamp_t7,embryo_Name_t8,embryo_Time_t8,embryo_Value_t8,embryo_Timestamp_t8,embryo_Name_t9,embryo_Time_t9,embryo_Value_t9,embryo_Timestamp_t9,embryo_Name_tB,embryo_Time_tB,embryo_Value_tB,embryo_Timestamp_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Value_tEB,embryo_Timestamp_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Value_tHB,embryo_Timestamp_tHB,embryo_Name_tM,embryo_Time_tM,embryo_Value_tM,embryo_Timestamp_tM,embryo_Name_tPB2,embryo_Time_tPB2,embryo_Value_tPB2,embryo_Timestamp_tPB2,embryo_Name_tPNa,embryo_Time_tPNa,embryo_Value_tPNa,embryo_Timestamp_tPNa,embryo_Name_tPNf,embryo_Time_tPNf,embryo_Value_tPNf,embryo_Timestamp_tPNf,embryo_Name_tSB,embryo_Time_tSB,embryo_Value_tSB,embryo_Timestamp_tSB,embryo_Name_tSC,embryo_Time_tSC,embryo_Value_tSC,embryo_Timestamp_tSC,idascore_IDAScore,idascore_IDATime,idascore_IDAVersion,AgeAtFertilization
0,827681,827682,PC1P7BHG_45418.6261401968,"PEDROSO, EDISON O. S.",06/05/1980,1980-05-06,1980,Ibirapuera,2024 - 1933,D2024.11.24_S03926_I3166_P-3,2024-11-24,3,6,3166,3,2024-11-24 12:50:00,None,Avoid,None,AB3,0,2024-11-30,KIDScoreD5 v3,ADMIN,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,FRAG2CAT,30.5,0-10%,2024.11.25 19:21:43,None,NaN,None,N

In [51]:
# embryoscope_embrioes_data.loc[embryoscope_embrioes_data['embryo_FertilizationTime'].isin(['2024-01-15 12:45:00'])].sort_values(by='embryo_embryo_number')

### 9. Embryoscope Clinisys Combined


In [52]:
print("=== EMBRYOSCOPE CLINISYS COMBINED DATA ===")
combined_data = results['embryoscope_clinisys_combined']
print(f"Records: {len(combined_data)}")
if not combined_data.empty:
    display(combined_data.head(20))
else:
    print("No embryoscope clinisys combined data found.")


=== EMBRYOSCOPE CLINISYS COMBINED DATA ===
Records: 31


,oocito_id,oocito_id_micromanipulacao,oocito_TCD,oocito_embryo_number,oocito_OrigemOocito,micro_numero_caso,micro_prontuario,micro_Data_DL,micro_data_procedimento,micro_oocitos,trat1_id,trat1_tentativa,trat1_tipo_procedimento,trat1_data_inicio_inducao,trat1_motivo_nao_transferir,trat1_data_transferencia,trat1_resultado_tratamento,trat1_fator_infertilidade1,trat1_origem_ovulo,trat1_bmi,trat1_previous_et,trat1_previous_et_od,emb_cong_id,emb_cong_qualidade,emb_cong_transferidos,emb_cong_colapso,cong_em_id,cong_em_CodCongelamento,cong_em_Data,descong_em_id,descong_em_CodDescongelamento,descong_em_DataTransferencia,descong_em_DataCongelamento,descong_em_DataDescongelamento,trat2_id,trat2_tentativa,trat2_tipo_procedimento,trat2_data_inicio_inducao,trat2_motivo_nao_transferir,trat2_data_transferencia,trat2_resultado_tratamento,trat2_bmi,trat2_previous_et,trat2_previous_et_od,prontuario,patient_PatientID,patient_PatientIDx,patient_FirstName,patient_LastName,patient_DateOfBirth,patient_YearOfBirth,patient_unit_huntington,treatment_TreatmentName,embryo_EmbryoID,embryo_EmbryoDate,embryo_embryo_number,embryo_Position,embryo_InstrumentNumber,embryo_WellNumber,embryo_FertilizationTime,embryo_FertilizationMethod,embryo_EmbryoFate,embryo_Description,embryo_EmbryoDescriptionID,embryo_KIDScore,embryo_KIDDate,embryo_KIDVersion,embryo_KIDUser,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Value_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Value_EVEN2,embryo_Timestamp_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Value_EVEN4,embryo_Timestamp_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Value_EVEN8,embryo_Timestamp_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Value_FRAG2,embryo_Timestamp_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Value_FRAG4,embryo_Timestamp_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Value_FRAG8,embryo_Timestamp_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Value_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Value_ICM,embryo_Timestamp_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Value_Line,embryo_Timestamp_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Value_MN2Type,embryo_Timestamp_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Value_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Value_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Value_PN,embryo_Timestamp_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Value_Pulsing,embryo_Timestamp_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Value_Strings,embryo_Timestamp_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Value_TE,embryo_Timestamp_TE,embryo_Name_t2,embryo_Time_t2,embryo_Value_t2,embryo_Timestamp_t2,embryo_Name_t3,embryo_Time_t3,embryo_Value_t3,embryo_Timestamp_t3,embryo_Name_t4,embryo_Time_t4,embryo_Value_t4,embryo_Timestamp_t4,embryo_Name_t5,embryo_Time_t5,embryo_Value_t5,embryo_Timestamp_t5,embryo_Name_t6,embryo_Time_t6,embryo_Value_t6,embryo_Timestamp_t6,embryo_Name_t7,embryo_Time_t7,embryo_Value_t7,embryo_Timestamp_t7,embryo_Name_t8,embryo_Time_t8,embryo_Value_t8,embryo_Timestamp_t8,embryo_Name_t9,embryo_Time_t9,embryo_Value_t9,embryo_Timestamp_t9,embryo_Name_tB,embryo_Time_tB,embryo_Value_tB,embryo_Timestamp_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Value_tEB,embryo_Timestamp_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Value_tHB,embryo_Timestamp_tHB,embryo_Name_tM,embryo_Time_tM,e

In [53]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'oocito_id_micromanipulacao', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'treatment_TreatmentName', 'embryo_EmbryoID', 'embryo_KIDScore', 'embryo_Description', 'embryo_EmbryoDescriptionID', 'embryo_embryo_number',
    'emb_cong_transferidos', 'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento', 'trat_tentativa'
]

# combined_data.loc[combined_data['oocito_id_micromanipulacao'].isin([21961]), colunas]
# combined_data.loc[combined_data['embryo_EmbryoDescriptionID'].isin(['AA5']), colunas] 
# combined_data.loc[:, colunas]

### 11. Planilha Embriologia FESH


In [54]:
print("=== PLANILHA EMBRIOLOGIA DATA FRESH===")
planilha_data = results['planilha_embriologia_fresh']
print(f"Records: {len(planilha_data)}")
if not planilha_data.empty:
    display(planilha_data)
else:
    print("No planilha embriologia data found.")


=== PLANILHA EMBRIOLOGIA DATA FRESH===
Records: 0
No planilha embriologia data found.


### 12. Planilha Embriologia FET


In [55]:
print("=== PLANILHA EMBRIOLOGIA DATA FET===")
planilha_data = results['planilha_embriologia_fet']
print(f"Records: {len(planilha_data)}")
if not planilha_data.empty:
    display(planilha_data)
else:
    print("No planilha embriologia data found.")


=== PLANILHA EMBRIOLOGIA DATA FET===
Records: 2


,pin,tipo_1,tipo_de_fet,tipo_biopsia,tipo_da_doacao,data_da_fet,idade_mulher,idade_do_cong_de_embriao,preparo_para_transferencia,dia_cryo,data_crio,dia_et,result,tipo_do_resultado,obs,gravidez_clinica,no_nascidos,gravidez_bioquimica,no_da_transfer_1a_2a_3a,no_et,tipo_de_tratamento,file_name,sheet_name,prontuario
0,827682,None,CYCLE CONTINUATION,PGT-A,None,2024-12-13,44,NaN,HORMONAL REPLACEMENT CYCLE,5 e 6,2024-11-30,6,EMBRYO TRANSFER,POSITIVO,None,1,None,1,1,1,FET FROM PGT,CASOS 2024 IBIRA.xlsx,FET,827681
1,827682,FET FROM FOT,THAWING OF EMBRYO,PGT-A,None,2025-09-13,45,NaN,None,5 e 6,2024-11-30,6,EMBRYO TRANSFER,POSITIVO,UTERO DE SUBSTITUIÇÃO EM YARA TAIZA BELCHIOR PIN 836751,1,None,None,None,<NA>,None,CASOS 2025 IBI.xlsx,FET,827681


### 13. Planilha Embryoscope Combined


In [56]:
print("=== PLANILHA EMBRYOSCOPE COMBINED DATA ===")
combined_planilha_data = results['planilha_embryoscope_combined']
print(f"Records: {len(combined_planilha_data)}")
if not combined_planilha_data.empty:
    display(combined_planilha_data.head(20))
else:
    print("No planilha embryoscope combined data found.")


=== PLANILHA EMBRYOSCOPE COMBINED DATA ===
Records: 31


,oocito_id,oocito_id_micromanipulacao,oocito_TCD,oocito_embryo_number,oocito_OrigemOocito,micro_numero_caso,micro_prontuario,micro_Data_DL,micro_data_procedimento,micro_oocitos,trat1_id,trat1_tentativa,trat1_tipo_procedimento,trat1_data_inicio_inducao,trat1_motivo_nao_transferir,trat1_data_transferencia,trat1_resultado_tratamento,trat1_fator_infertilidade1,trat1_origem_ovulo,trat1_bmi,trat1_previous_et,trat1_previous_et_od,emb_cong_id,emb_cong_qualidade,emb_cong_transferidos,emb_cong_colapso,cong_em_id,cong_em_CodCongelamento,cong_em_Data,descong_em_id,descong_em_CodDescongelamento,descong_em_DataTransferencia,descong_em_DataCongelamento,descong_em_DataDescongelamento,trat2_id,trat2_tentativa,trat2_tipo_procedimento,trat2_data_inicio_inducao,trat2_motivo_nao_transferir,trat2_data_transferencia,trat2_resultado_tratamento,trat2_bmi,trat2_previous_et,trat2_previous_et_od,prontuario,patient_PatientID,patient_PatientIDx,patient_FirstName,patient_LastName,patient_DateOfBirth,patient_YearOfBirth,patient_unit_huntington,treatment_TreatmentName,embryo_EmbryoID,embryo_EmbryoDate,embryo_embryo_number,embryo_Position,embryo_InstrumentNumber,embryo_WellNumber,embryo_FertilizationTime,embryo_FertilizationMethod,embryo_EmbryoFate,embryo_Description,embryo_EmbryoDescriptionID,embryo_KIDScore,embryo_KIDDate,embryo_KIDVersion,embryo_KIDUser,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Value_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Value_EVEN2,embryo_Timestamp_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Value_EVEN4,embryo_Timestamp_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Value_EVEN8,embryo_Timestamp_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Value_FRAG2,embryo_Timestamp_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Value_FRAG4,embryo_Timestamp_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Value_FRAG8,embryo_Timestamp_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Value_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Value_ICM,embryo_Timestamp_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Value_Line,embryo_Timestamp_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Value_MN2Type,embryo_Timestamp_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Value_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Value_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Value_PN,embryo_Timestamp_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Value_Pulsing,embryo_Timestamp_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Value_Strings,embryo_Timestamp_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Value_TE,embryo_Timestamp_TE,embryo_Name_t2,embryo_Time_t2,embryo_Value_t2,embryo_Timestamp_t2,embryo_Name_t3,embryo_Time_t3,embryo_Value_t3,embryo_Timestamp_t3,embryo_Name_t4,embryo_Time_t4,embryo_Value_t4,embryo_Timestamp_t4,embryo_Name_t5,embryo_Time_t5,embryo_Value_t5,embryo_Timestamp_t5,embryo_Name_t6,embryo_Time_t6,embryo_Value_t6,embryo_Timestamp_t6,embryo_Name_t7,embryo_Time_t7,embryo_Value_t7,embryo_Timestamp_t7,embryo_Name_t8,embryo_Time_t8,embryo_Value_t8,embryo_Timestamp_t8,embryo_Name_t9,embryo_Time_t9,embryo_Value_t9,embryo_Timestamp_t9,embryo_Name_tB,embryo_Time_tB,embryo_Value_tB,embryo_Timestamp_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Value_tEB,embryo_Timestamp_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Value_tHB,embryo_Timestamp_tHB,embryo_Name_tM,embryo_Time_tM,e

In [57]:
cols = [
    'oocito_id', 'oocito_id_micromanipulacao', 'oocito_TCD', 'oocito_embryo_number', 
    'oocito_OrigemOocito', 'micro_numero_caso', 'micro_prontuario', 'micro_Data_DL', 
    'micro_data_procedimento', 'micro_oocitos', 'trat1_id', 'trat1_tentativa', 
    'trat1_tipo_procedimento', 'trat1_data_inicio_inducao', 'trat1_motivo_nao_transferir', 
    'trat1_data_transferencia', 'trat1_resultado_tratamento', 'trat1_fator_infertilidade1', 
    'trat1_origem_ovulo', 'trat1_bmi', 'trat1_previous_et', 'trat1_previous_et_od', 
    'emb_cong_id', 'emb_cong_qualidade', 'emb_cong_transferidos', 'cong_em_id', 
    'cong_em_CodCongelamento', 'cong_em_Data', 'descong_em_id', 'descong_em_CodDescongelamento', 
    'descong_em_DataTransferencia', 'descong_em_DataCongelamento', 'descong_em_DataDescongelamento', 
    'trat2_id', 'trat2_tentativa', 'trat2_tipo_procedimento', 'trat2_data_inicio_inducao', 
    'trat2_motivo_nao_transferir', 'trat2_data_transferencia', 'trat2_resultado_tratamento', 
    'trat2_bmi', 'trat2_previous_et', 'trat2_previous_et_od', 'prontuario', 
    'patient_PatientID', 'patient_PatientIDx', 'join_step', 'matched_planilha_prontuario', 
    'matched_planilha_transfer_date', 'merged_numero_de_nascidos', 'incubadora_padronizada', 
    'outcome', 'outcome_type', 'chart_or_pin', 'date_of_delivery', 
    'gestational_age_at_delivery', 'type_of_delivery', 'baby_1_weight', 'baby_2_weight', 
    'baby_3_weight', 'complications_of_pregnancy_specify', 'number_of_fet_after_originally_frozen', 
    'n_of_biopsied', 'n_of_normal', 'number_of_embryos_transferred', 'year', 'unidade', 
    'fresh_fator_1', 'fresh_incubadora', 'fresh_data_crio', 'fresh_tipo_1', 
    'fresh_tipo_de_inseminacao', 'fresh_tipo_biopsia', 'fresh_altura', 'fresh_peso', 
    'fresh_data_de_nasc', 'fresh_idade_espermatozoide', 'fresh_origem', 'fresh_tipo', 
    'fresh_opu', 'fresh_total_de_mii', 'fresh_qtd_blasto', 'fresh_qtd_blasto_tq_a_e_b', 
    'fresh_no_biopsiados', 'fresh_qtd_analisados', 'fresh_qtd_normais', 'fresh_dia_cryo', 
    'fresh_file_name', 'fresh_sheet_name', 'fet_data_crio', 'fet_resultado', 
    'fet_tipo_resultado', 'fet_tipo_1', 'fet_tipo_de_tratamento', 'fet_tipo_de_fet', 
    'fet_tipo_biopsia', 'fet_tipo_da_doacao', 'fet_idade_mulher', 'fet_idade_do_cong_de_embriao', 
    'fet_preparo_para_transferencia', 'fet_dia_cryo', 'fet_no_da_transfer_1a_2a_3a', 
    'fet_dia_et', 'fet_no_et', 'fet_gravidez_bioquimica', 'fet_gravidez_clinica', 
    'fet_file_name', 'fet_sheet_name'
]

combined_planilha_data[cols]

,oocito_id,oocito_id_micromanipulacao,oocito_TCD,oocito_embryo_number,oocito_OrigemOocito,micro_numero_caso,micro_prontuario,micro_Data_DL,micro_data_procedimento,micro_oocitos,trat1_id,trat1_tentativa,trat1_tipo_procedimento,trat1_data_inicio_inducao,trat1_motivo_nao_transferir,trat1_data_transferencia,trat1_resultado_tratamento,trat1_fator_infertilidade1,trat1_origem_ovulo,trat1_bmi,trat1_previous_et,trat1_previous_et_od,emb_cong_id,emb_cong_qualidade,emb_cong_transferidos,cong_em_id,cong_em_CodCongelamento,cong_em_Data,descong_em_id,descong_em_CodDescongelamento,descong_em_DataTransferencia,descong_em_DataCongelamento,descong_em_DataDescongelamento,trat2_id,trat2_tentativa,trat2_tipo_procedimento,trat2_data_inicio_inducao,trat2_motivo_nao_transferir,trat2_data_transferencia,trat2_resultado_tratamento,trat2_bmi,trat2_previous_et,trat2_previous_et_od,prontuario,patient_PatientID,patient_PatientIDx,join_step,matched_planilha_prontuario,matched_planilha_transfer_date,merged_numero_de_nascidos,incubadora_padronizada,outcome,outcome_type,chart_or_pin,date_of_delivery,gestational_age_at_delivery,type_of_delivery,baby_1_weight,baby_2_weight,baby_3_weight,complications_of_pregnancy_specify,number_of_fet_after_originally_frozen,n_of_biopsied,n_of_normal,number_of_embryos_transferred,year,unidade,fresh_fator_1,fresh_incubadora,fresh_data_crio,fresh_tipo_1,fresh_tipo_de_inseminacao,fresh_tipo_biopsia,fresh_altura,fresh_peso,fresh_data_de_nasc,fresh_idade_espermatozoide,fresh_origem,fresh_tipo,fresh_opu,fresh_total_de_mii,fresh_qtd_blasto,fresh_qtd_blasto_tq_a_e_b,fresh_no_biopsiados,fresh_qtd_analisados,fresh_qtd_normais,fresh_dia_cryo,fresh_file_name,fresh_sheet_name,fet_data_crio,fet_resultado,fet_tipo_resultado,fet_tipo_1,fet_tipo_de_tratamento,fet_tipo_de_fet,fet_tipo_biopsia,fet_tipo_da_doacao,fet_idade_mulher,fet_idade_do_cong_de_embriao,fet_preparo_para_transferencia,fet_dia_cryo,fet_no_da_transfer_1a_2a_3a,fet_dia_et,fet_no_et,fet_gravidez_bioquimica,fet_gravidez_clinica,fet_file_name,fet_sheet_name
0,228983,23412,Descartado,8,Descongelado OR,ED401/24,827681,2024-11-24,2024-11-24,8,<NA>,None,None,NaT,None,NaT,None,None,None,NaN,<NA>,<NA>,<NA>,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,827681,827682,PC1P7BHG_45418.6261401968,<NA>,<NA>,NaT,<NA>,None,None,None,None,NaT,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,NaT,None,None,None,NaN,<NA>,NaT,None,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,None,None,NaT,None,None,None,None,None,None,None,<NA>,NaN,None,None,None,<NA>,<NA>,None,None,None,None
1,228982,23412,Criopreservado,7,Descongelado OR,ED401/24,827681,2024-11-24,2024-11-24,8,<NA>,None,None,NaT,None,NaT,None,None,None,NaN,<NA>,<NA>,79828,Blastocisto Grau 3 - B - C,Transferido,27413,E3168/24,2024-11-30,18866,2083/25,2025-09-13,2024-11-30,2025-09-13,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,827681,827682,PC1P7BHG_45418.6261401968,2,827681,2025-09-13,<NA>,None,None,None,None,NaT,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,NaT,None,None,None,NaN,<NA>,NaT,None,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,None,None,2024-11-30,EMBRYO TRANSFER,POSITIVO,FET FROM FOT,None,THAWING OF EMBRYO,PGT-A,None,45,NaN,None,5 e 6,None,6,<NA>,None,1,CASOS 2025 IBI.xlsx,FET
2,228981,23412,Criopreservado,6,Descongelado OR,ED401/24,827681,2024-11-24,2024-11-24,8,<NA>,None,None,NaT,None,NaT,None,None,None,NaN,<NA>,<NA>,79827,Blastocisto Grau 5 - A - B,Transferido,27413,E3168/24,2024-11-30,16635,2760/24,2024-12-13,2024-11-30,2024-12-13,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,827681,827682,PC1P7BHG_45418.6261401968,2,827681,2024-12-13,1,None,Embryo transfer,Delivery (> 21 WA),827682.0,NaT,None,None,None,None,None,None,1st transfer,None,None,1.0,2024,Ibirapuera,None,None,NaT,None,None,None,NaN,<NA>,NaT,None,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,None,None,2024-11-30,EMBRYO TRANSFER,POSITIVO,None,FET FROM PGT,CYCLE CONTINUAT

### 14. Congelamentos Ovulos


In [58]:
print("=== CONGELAMENTOS OVULOS DATA ===")
congelamentos_ovulos_data = results['congelamentos_ovulos']
print(f"Records: {len(congelamentos_ovulos_data)}")
if not congelamentos_ovulos_data.empty:
    display(congelamentos_ovulos_data)
else:
    print("No congelamentos ovulos data found.")


=== CONGELAMENTOS OVULOS DATA ===
Records: 1


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,condicoes_amostra,empresa_transporte,clinica_origem,responsavel_recebimento,responsavel_recebimento_data,responsavel_armazenamento,responsavel_armazenamento_data,NOvulos,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Motivo,Observacoes,BiologoResponsavel,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,13198,O1309/24,1,827681,esposa,2024-07-16,NaT,-,CONFORME,FAIRFAX,OVOBANK,4005,2024-10-30,4005,2024-10-30,8,4,OD-EGG1526 / OVOAR093926-2,None,None,None,Vitrificação,Ciclo,VINDOS DA OVOBANK EM 30/10/2024.\r\nID: OVOAR093926-2,<NA>,Bruna Lázaro Lourenço,Mariana Nicolielo,Cobrar,8dbe355a2c219fdcc8aea1ead686b794,2025-07-21 21:38:49


### 15. Descongelamentos Ovulos


In [59]:
print("=== DESCONGELAMENTOS OVULOS DATA ===")
descongelamentos_ovulos_data = results['descongelamentos_ovulos']
print(f"Records: {len(descongelamentos_ovulos_data)}")
if not descongelamentos_ovulos_data.empty:
    display(descongelamentos_ovulos_data)
else:
    print("No descongelamentos ovulos data found.")


=== DESCONGELAMENTOS OVULOS DATA ===
Records: 2


,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp
0,4255,DO665/24,1,827681,0,2024-07-16,2024-11-24,-,OD-EGG1526 / OVOAR093926-2,O1309/24,AI,1,4,Desvitrificação,Descongelados 8 óvulos doados ODEGG1526/ OVOAR093926-2,Izadora Reis,Bruna Lázaro Lourenço,57b9167b3b73821ad7bcd73069fd4e7f,2025-07-21 21:38:53
1,3846,DO267/24,1,827681,786406,2024-05-06,2024-05-07,52/24,OVOD2335,O86/24,A3,1,3,Desvitrificação,None,None,None,e5edd44a3c7390113d57d4b4a1712d71,2025-07-21 21:38:53


### 16. Ovulos Congelados


In [60]:
print("=== OVULOS CONGELADOS DATA ===")
ovulos_congelados_data = results['ovulos_congelados']
print(f"Records: {len(ovulos_congelados_data)}")
if not ovulos_congelados_data.empty:
    display(ovulos_congelados_data)
else:
    print("No ovulos congelados data found.")


=== OVULOS CONGELADOS DATA ===
Records: 8


,id,id_congelamento,id_descongelamento,prontuario,pailletes,cores,ovulo,celulas,qualidade,qualidade_recongelamento,comentarios,comentarios_descongelamento,rc,dia2,maturidade,dia2_2,dia2_3,clivou_d2,transferidos,pgd,resultado_pgd,tanque_amostra,caneca_amostra,rack_amostra,observacao,destino,hash,extraction_timestamp
0,92008,13198,4255,827681,P4,<NA>,8.0,<NA>,<NA>,<NA>,PALHETA LARANJA - VINDOS DA OVOBANK,None,None,MII,MII,None,None,<NA>,None,<NA>,<NA>,AI,2,OD-EGG1526 / OVOAR093926-2,VINDOS DA OVOBANK,<NA>,b222ff7030a2cf219ee82d9326620b5e,2025-10-14 21:04:28
1,92007,13198,4255,827681,P4,<NA>,7.0,<NA>,<NA>,<NA>,PALHETA LARANJA - VINDOS DA OVOBANK,None,None,MII,MII,None,None,<NA>,None,<NA>,<NA>,AI,2,OD-EGG1526 / OVOAR093926-2,VINDOS DA OVOBANK,<NA>,5ac12deeb310d8a5925b17b23b6eeecb,2025-10-14 21:04:28
2,92006,13198,4255,827681,P3,<NA>,6.0,<NA>,<NA>,<NA>,PALHETA LARANJA - VINDOS DA OVOBANK,None,None,MII,MII,None,None,<NA>,None,<NA>,<NA>,AI,2,OD-EGG1526 / OVOAR093926-2,VINDOS DA OVOBANK,<NA>,611c78bbbd49e744896a71dbb3e3542a,2025-10-14 21:04:28
3,92005,13198,4255,827681,P3,<NA>,5.0,<NA>,<NA>,<NA>,PALHETA LARANJA - VINDOS DA OVOBANK,None,None,MII,MII,None,None,<NA>,None,<NA>,<NA>,AI,2,OD-EGG1526 / OVOAR093926-2,VINDOS DA OVOBANK,<NA>,a7d1aaebfe969f2186eb50cf129080af,2025-10-14 21:04:28
4,92004,13198,4255,827681,P2,<NA>,4.0,<NA>,<NA>,<NA>,PALHETA LARANJA - VINDOS DA OVOBANK,None,None,MII,MII,None,None,<NA>,None,<NA>,<NA>,AI,2,OD-EGG1526 / OVOAR093926-2,VINDOS DA OVOBANK,<NA>,619a65d6407ce4188400ee2273fdcb51,2025-10-14 21:04:28
5,92003,13198,4255,827681,P2,<NA>,3.0,<NA>,<NA>,<NA>,PALHETA LARANJA - VINDOS DA OVOBANK,None,None,MII,MII,None,None,<NA>,None,<NA>,<NA>,AI,2,OD-EGG1526 / OVOAR093926-2,VINDOS DA OVOBANK,<NA>,7dd3a7012a6ae527db9e21f6047be019,2025-10-14 21:04:28
6,92002,13198,4255,827681,P1,<NA>,2.0,<NA>,<NA>,<NA>,PALHETA LARANJA - VINDOS DA OVOBANK,None,None,MII,MII,None,None,<NA>,None,<NA>,<NA>,AI,2,OD-EGG1526 / OVOAR093926-2,VINDOS DA OVOBANK,<NA>,9dc75ec8ea18436e2bf709fd6f5a9e13,2025-10-14 21:04:28
7,92001,13198,4255,827681,P1,<NA>,1.0,<NA>,<NA>,<NA>,PALHETA LARANJA - VINDOS DA OVOBANK,None,None,MII,MII,None,None,<NA>,None,<NA>,<NA>,AI,2,OD-EGG1526 / OVOAR093926-2,VINDOS DA OVOBANK,<NA>,a7d4357030cc785f3c0ee92060fb8e46,2025-10-14 21:04:28


### 17. Redlara 


In [61]:
print("=== Redlara Unified ===")
redlara_unified = results['redlara_unified']
print(f"Records: {len(redlara_unified)}")
if not redlara_unified.empty:
    display(redlara_unified)
else:
    print("No redlara unified data found.")


=== Redlara Unified ===
Records: 1


,outcome,outcome_type,chart_or_pin,date_when_embryos_were_cryopreserved,date_of_embryo_transfer,date_of_delivery,gestational_age_at_delivery,type_of_delivery,number_of_newborns,baby_1_weight,baby_2_weight,baby_3_weight,complications_of_pregnancy_specify,number_of_fet_after_originally_frozen,n_of_biopsied,n_of_normal,number_of_embryos_transferred,year,unidade,prontuario
0,Embryo transfer,Delivery (> 21 WA),827682.0,2024-11-30,2024-12-13,NaT,None,None,1,None,None,None,None,1st transfer,None,None,1.0,2024,Ibirapuera,827681


### 18. Redlara - planilha combined


In [62]:
print("=== Redlara planilha combined ===")
redlara_planilha_combined = results['redlara_planilha_combined']
print(f"Records: {len(redlara_planilha_combined)}")
if not redlara_planilha_combined.empty:
    display(redlara_planilha_combined)
else:
    print("No redlara_planilha_combined data found.")

=== Redlara planilha combined ===
Records: 2


,prontuario,transfer_date,merged_numero_de_nascidos,incubadora_padronizada,outcome,outcome_type,chart_or_pin,date_when_embryos_were_cryopreserved,date_of_delivery,gestational_age_at_delivery,type_of_delivery,baby_1_weight,baby_2_weight,baby_3_weight,complications_of_pregnancy_specify,number_of_fet_after_originally_frozen,n_of_biopsied,n_of_normal,number_of_embryos_transferred,year,unidade,fresh_data_da_puncao,fresh_fator_1,fresh_incubadora,fresh_data_crio,fresh_tipo_1,fresh_tipo_de_inseminacao,fresh_tipo_biopsia,fresh_altura,fresh_peso,fresh_data_de_nasc,fresh_idade_espermatozoide,fresh_origem,fresh_tipo,fresh_opu,fresh_total_de_mii,fresh_qtd_blasto,fresh_qtd_blasto_tq_a_e_b,fresh_no_biopsiados,fresh_qtd_analisados,fresh_qtd_normais,fresh_dia_cryo,fresh_file_name,fresh_sheet_name,fet_data_crio,fet_resultado,fet_tipo_resultado,fet_tipo_1,fet_tipo_de_tratamento,fet_tipo_de_fet,fet_tipo_biopsia,fet_tipo_da_doacao,fet_idade_mulher,fet_idade_do_cong_de_embriao,fet_preparo_para_transferencia,fet_dia_cryo,fet_no_da_transfer_1a_2a_3a,fet_dia_et,fet_no_et,fet_gravidez_bioquimica,fet_gravidez_clinica,fet_file_name,fet_sheet_name
0,827681,2024-12-13,1,None,Embryo transfer,Delivery (> 21 WA),827682.0,2024-11-30,NaT,None,None,None,None,None,None,1st transfer,None,None,1.0,2024,Ibirapuera,NaT,None,None,NaT,None,None,None,NaN,<NA>,NaT,None,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,None,None,2024-11-30,EMBRYO TRANSFER,POSITIVO,None,FET FROM PGT,CYCLE CONTINUATION,PGT-A,None,44,NaN,HORMONAL REPLACEMENT CYCLE,5 e 6,1,6,1,1,1,CASOS 2024 IBIRA.xlsx,FET
1,827681,2025-09-13,<NA>,None,None,None,None,NaT,NaT,None,None,None,None,None,None,None,None,None,None,<NA>,None,NaT,None,None,NaT,None,None,None,NaN,<NA>,NaT,None,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,None,None,None,2024-11-30,EMBRYO TRANSFER,POSITIVO,FET FROM FOT,None,THAWING OF EMBRYO,PGT-A,None,45,NaN,None,5 e 6,None,6,<NA>,None,1,CASOS 2025 IBI.xlsx,FET
